# MSDS 7331 - Mini-Project 1
## Jeff Weltman, Kevin Dickens, Jordan Kassof
## June 10, 2018

<hr>

# Introduction

**Primer:** We are using the Belk North Carolina Education Dataset (available on Github here) to explore the factors involved when a school outperforms, on average, 

In Lab 1, we explored the relationships in our data using a wide range of st

# Section 1: Create Models

**50	Create a logistic regression model and a support vector machine model for the classification task involved with your dataset. Assess how well each model performs (use 80/20 training/testing split for your data). Adjust parameters of the models to make them more accurate. If your dataset size requires the use of stochastic gradient descent, then linear kernel only is fine to use. That is, the SGDClassifier is fine to use for optimizing logistic regression and linear support vector machines. For many problems, SGD will be required in order to train the SVM model in a reasonable timeframe**

In this section, we'll use two classical data mining techniques to further our understanding of the relationships found in our data.

## Section 1A: Logistic Regression

**Some Introductory commentary on logistic regression goes here **

In [1]:
# Place holder for code or markdown related to the 
# generation and optimization of a logistic regression model 

In [2]:
# Place holder for code or markdown related to the 
# generation and optimization of a logistic regression model 

In [3]:
# Place holder for code or markdown related to the 
# generation and optimization of a logistic regression model 

In [4]:
# Place holder for code or markdown related to the 
# generation and optimization of a logistic regression model 

In [5]:
# Place holder for code or markdown related to the 
# generation and optimization of a logistic regression model 

In [6]:
# Place holder for code or markdown related to the 
# generation and optimization of a logistic regression model 

## Section 1B: Support Vector Machines

**Some Introductory commentary on logistic regression goes here **

In [7]:
# Place holder for code or markdown related to the 
# generation and optimization of a support vector machine

In [8]:
# Place holder for code or markdown related to the 
# generation and optimization of a support vector machine

In [9]:
# Place holder for code or markdown related to the 
# generation and optimization of a support vector machine

In [10]:
# Place holder for code or markdown related to the 
# generation and optimization of a support vector machine

In [11]:
# Place holder for code or markdown related to the 
# generation and optimization of a support vector machine

In [12]:
# Place holder for code or markdown related to the 
# generation and optimization of a support vector machine

# Section 2: Model Advantages

**Discuss the advantages of each model for each classification task. Does one type of model offer superior performance over another in terms of prediction accuracy? In terms of training time or efficiency? Explain in detail.**

In this section we will explore the nature of logistic regression and SVMs, including pros, cons, and other conisderations.

In [13]:
## Placeholder for code or markdwon related to compare & contrast of the two types of models

In [14]:
## Placeholder for code or markdwon related to compare & contrast of the two types of models

In [15]:
## Placeholder for code or markdwon related to compare & contrast of the two types of models

# Section 3: Interpret Feature Importance

**30	Use the weights from logistic regression to interpret the importance of different features for the classification task. Explain your interpretation in detail. Why do you think some variables are more important?** 

In [16]:
## Placeholder for code or markdwon related to section 3

In [17]:
## Placeholder for code or markdwon related to section 3

In [18]:
## Placeholder for code or markdwon related to section 3

# Section 4: Interpret Feature Importance

**	10	Look at the chosen support vectors for the classification task. Do these provide any insight into the data? Explain. If you used stochastic gradient descent (and therefore did not explicitly solve for support vectors), try subsampling your data to train the SVC model— then analyze the support vectors from the subsampled dataset**

In [19]:
# Placeholder for code or markdown related to section 4

In [20]:
# Placeholder for code or markdown related to section 4

In [ ]:
# Placeholder for code or markdown related to section 4